In [1]:
import pandas as pd

origin = pd.read_csv('/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv', parse_dates=['created_at'])


In [222]:
from keras.utils.np_utils import to_categorical
GOOD_THRESHOLD = 100

def prepareData(df):    
    good = df[df["num_points"] >= GOOD_THRESHOLD]
    bad = df[df["num_points"] < GOOD_THRESHOLD]
    bad = bad.sample(n=good.shape[0])
    data = good.append(bad)
    data = data.sample(frac=1).reset_index(drop=True)
    
    dayofweek = data['created_at'].dt.dayofweek.values[:,np.newaxis]
    hour = data['created_at'].dt.hour.values[:,np.newaxis]
    month = data['created_at'].dt.month.values[:,np.newaxis]
    day = data['created_at'].dt.day.values[:,np.newaxis]
    
    dayofweek = dayofweek / dayofweek.max() 
    hour = hour / hour.max()
    month = month / month.max()
    day = day / day.max()
    
    X = np.hstack((dayofweek, hour, month, day))

    label = data["num_points"].apply(lambda x: 1 if x >= GOOD_THRESHOLD else 0)
    y = to_categorical(label)
    
    #return df['created_at'], y, df["num_points"]

    return X, y, label

In [223]:
data = origin[['id', 'num_points', 'created_at']]

train = data.sample(frac=0.8)
test = data.drop(train.index)

X_train, y_train, y2_train = prepareData(train)
X_test, y_test, y2_test = prepareData(test)

In [218]:
import xgboost as xgb

params = {"objective": "binary:logistic",
          "eta": 0.15,
          "max_depth": 20,
          "min_child_weight": 10,
          "silent": 1,
          "subsample": 0.7,
          "colsample_bytree": 0.7,
          "seed": 1337}
num_trees=1001
gbm = xgb.train(params, xgb.DMatrix(X_train, y2_train), num_trees)

from sklearn.metrics import precision_score, recall_score

res = gbm.predict(xgb.DMatrix(X_test))

print(res)
res2 = np.zeros(len(res), dtype=int)
for i in range(0, len(res)):
    res2[i] = 1 if res[i] >= 0.5 else 0
print(res2)

precision = precision_score(y2_test, res2)
recall = recall_score(y2_test, res2)
print(precision)
print(recall)

[ 0.04500242  0.06725914  0.02041602 ...,  0.03572112  0.02655071
  0.05526261]
[0 0 0 ..., 0 0 0]
0.0
0.0


/Users/Hoiy/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [211]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, criterion="entropy", random_state=1)
rf.fit(X_train, y2_train)

res = rf.predict(X_test)

#validate_2(y_test, res)
precision = precision_score(y2_test, res)
recall = recall_score(y2_test, res)
print(precision)
print(recall)

0.528915662651
0.532121212121


In [226]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=10, random_state=1)
gbc.fit(X_train, y2_train)

res = gbc.predict(X_test)

precision = precision_score(y2_test, res)
recall = recall_score(y2_test, res)
print(precision)
print(recall)

0.509821038848
0.507164567955


In [252]:
from keras.layers import Input, LSTM, RepeatVector, Dense
from keras.models import Model, Sequential

predictor = Sequential()
predictor.add(Dense(10, activation='relu', input_shape=(4,)))
predictor.add(Dense(10, activation='relu'))
predictor.add(Dense(2, activation='softmax'))

In [253]:
predictor.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc', 'precision', 'recall','fmeasure'])

In [254]:
# happy learning!
predictor.fit(X_train, y_train, nb_epoch=10, batch_size=32, validation_split=0.2)

Train on 14779 samples, validate on 3695 samples
Epoch 1/10
14779/14779 [==============================] - 1s - loss: 0.6929 - acc: 0.5135 - precision: 0.5135 - recall: 0.5135 - fmeasure: 0.5135 - val_loss: 0.6919 - val_acc: 0.5204 - val_precision: 0.5204 - val_recall: 0.5204 - val_fmeasure: 0.5204
Epoch 2/10
14779/14779 [==============================] - 1s - loss: 0.6920 - acc: 0.5212 - precision: 0.5212 - recall: 0.5212 - fmeasure: 0.5212 - val_loss: 0.6921 - val_acc: 0.5277 - val_precision: 0.5277 - val_recall: 0.5277 - val_fmeasure: 0.5277
Epoch 3/10
14779/14779 [==============================] - 1s - loss: 0.6919 - acc: 0.5217 - precision: 0.5217 - recall: 0.5217 - fmeasure: 0.5217 - val_loss: 0.6918 - val_acc: 0.5240 - val_precision: 0.5240 - val_recall: 0.5240 - val_fmeasure: 0.5240
Epoch 4/10
14779/14779 [==============================] - 1s - loss: 0.6912 - acc: 0.5228 - precision: 0.5228 - recall: 0.5228 - fmeasure: 0.5228 - val_loss: 0.6921 - val_acc: 0.5267 - val_precision

In [255]:
import numpy as np

res = predictor.predict(X_test)

print(res)
res2 = np.zeros(len(res), dtype=int)
for i in range(0, len(res)):
    res2[i] = 1 if res[i][1] >= 0.5 else 0
print(res2)

precision = precision_score(y2_test, res2)
recall = recall_score(y2_test, res2)
print(precision)
print(recall)

[[ 0.41849762  0.58150238]
 [ 0.47418538  0.52581465]
 [ 0.47119316  0.52880687]
 ..., 
 [ 0.49984667  0.50015336]
 [ 0.49410808  0.50589192]
 [ 0.46115309  0.53884691]]
[1 1 1 ..., 1 1 1]
0.504700643246
0.885801128962
